In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import argparse
import pickle as pkl
import time
from copy import deepcopy
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from random import shuffle
import torch.nn.functional as F
from torchvision.transforms import v2
from torch import nn

c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, end_maxpool = False):
        super(ResidualBlock, self).__init__()
        if(downsample is not None):
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace=False),
                            nn.MaxPool2d(kernel_size=2, stride=2)
                            )  # Changed inplace to False
        else:
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False)
                            )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(out_channels),
                        nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False))  # Changed inplace to False
        self.downsample = downsample
        self.relu = nn.Hardtanh(min_val=-1.0, max_val=1.0, inplace=False)  # Changed inplace to False
        self.out_channels = out_channels
        self.end_maxpool = end_maxpool

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out = out + residual
        if self.end_maxpool:
            out = F.relu(out, inplace=False)
        else:
            out = F.hardtanh(out, inplace=False, min_val=-1.0, max_val=1.0)   # Use non-in-place ReLU
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2, in_chanels = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_chanels, 64, kernel_size = 7, stride = 1, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=False))
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2, end_maxpool = True)
        self.avgpool = nn.MaxPool2d(7, stride=1)
        self.fc = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, end_maxpool = False):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=1, padding='same'),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            if i == blocks-1 and end_maxpool:
                layers.append(block(self.inplanes, planes, end_maxpool = True))
            else:
                layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.hardtanh(x)
        x = self.fc2(x)
        return x

In [4]:
model_resnet = ResNet(ResidualBlock, [5, 6, 6, 4], num_classes = 101, in_chanels=6)

In [5]:
model_resnet=torch.load("best_resnet_nCaltech101_Hardtanh_ReLUmaxpool_EST_FC2_corrected_trilinear_98.pt", weights_only=False)
model_resnet.eval().to("cuda")
print(model_resnet)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Hardtanh(min_val=-1.0, max_val=1.0)
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Hardtanh(min_val=-1.0, max_val=1.0)
      )
      (relu): Hardtanh(min_val=-1.0, max_val=1.0)
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kerne

In [6]:
temp = torch.rand((1,6,224,224))
model_resnet(temp.to("cuda"))

tensor([[ 7.4549, -0.7699, -0.6710, -0.4772,  1.7812, -1.9952,  0.3704,  2.0780,
         -4.6632, -2.8911,  4.9264,  0.8885, -4.1747,  0.5351, -5.3044, -3.1034,
          3.0183, -0.6036,  2.3824, -5.8221, -0.9069,  1.5042, -2.7596, -0.8619,
         -1.9166,  3.4755,  4.1127, -2.6982, -1.6412, -0.7610,  3.1219,  0.9123,
          1.8907, -0.9885, -0.0906, -0.9517, -1.8468, -1.8142,  3.5716, -0.9091,
          0.1116, -1.3690,  0.7212, -3.0327, -5.8912,  1.2094, -0.4483,  4.2604,
         -0.6632,  0.9809, -5.0322, -0.6131, -0.1402,  2.5098, -3.4294,  2.3544,
          1.3008, -1.5584, -1.1215,  2.0692,  2.8084,  4.3361,  4.2195,  1.9735,
          4.1779, -5.7576,  2.3493,  0.1823, -6.4078,  2.0071, -4.2471,  2.9470,
          0.5222,  2.0050,  7.8277,  0.5656,  0.2018, -3.6788,  0.6687,  4.0763,
          1.4433, -1.9025,  0.5337, -2.4090, -0.5783,  1.0867,  1.8979, -2.9545,
          2.5140,  2.9677, -5.2192, -5.3157,  3.7737,  3.2428,  1.0753, -0.3422,
         -7.1640,  1.3967, -

In [7]:
from SNN.Check_Hardtanh import fuse_conv_and_bn
from SNN.Check_Hardtanh import SpikingConv2D
from SNN.Check_Hardtanh import MaxMinPool2D
from SNN.Check_Hardtanh import LayerSNN_all
from SNN.Check_Hardtanh import SpikingDense_positive_tanH, SpikingDense_all_all, SpikingDense

In [8]:
class ResNet_ttfs(nn.Module):
    def __init__(self, model : ResNet):
        super(ResNet_ttfs, self).__init__()
        model.eval()
        robustness_params={
            'noise':0.0,
            'time_bits':0,
            'weight_bits': 0,
            'latency_quantiles':0.0
        }
        model.to('cuda')
        conv_fused = fuse_conv_and_bn(model.conv1[0], model.conv1[1], device = 'cuda')
        self.conv_first = SpikingConv2D(64, "temp1", device = 'cuda', padding=(3,3), stride=2, kernel_size=(7,7),robustness_params=robustness_params, kernels=conv_fused.weight.data, biases= conv_fused.bias.data)
        max_vect = torch.tensor([1]*6)
        tmin, tmax, max_vect, scalar = self.conv_first.set_params(0.0,1.0,max_vect)
        self.tmin_post_pool = tmax
        self.pool = MaxMinPool2D(3, tmax.data,2,1).to("cuda")
        self.layer0SNN = LayerSNN_all(model.layer0, 64, 64, 5,device = 'cuda')
        tmax_prev = tmax
        tmin, tmax, max_vect, scalar = self.layer0SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer1SNN = LayerSNN_all(model.layer1, 64, 128, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer1SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer2SNN = LayerSNN_all(model.layer2, 128, 256, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer2SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer3SNN = LayerSNN_all(model.layer3, 256, 512, 4,device = 'cuda',end_maxpool=True)
        tmin, tmax, max_vect, scalar = self.layer3SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.pool2 = MaxMinPool2D(7, tmax.data,1,0).to("cuda")
        # temp = torch.ones((512,2,2))
        # max_vect = ((temp.T)*max_vect[:512]).T
        # max_vect = max_vect.view(-1)
        print(max_vect.shape)
        max_vect = (torch.ones([512, 1, 1]).to("cuda").T*max_vect[:512]).T.contiguous().view(512).to("cuda")

        weights = model.fc.weight.detach().clone()
        biases = model.fc.bias.detach().clone()
        self.layer_fc = SpikingDense_positive_tanH(256, 512,"test",robustness_params=robustness_params,device = 'cuda',weights=weights, biases=biases)
        
        tmin, tmax, max_vect, scalar = self.layer_fc.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer_fc2 = SpikingDense_all_all(101,256, '',model.fc2.weight, model.fc2.bias,robustness_params=robustness_params,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer_fc2.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.tmin, self.tmax = tmin, tmax
        self.scalar = scalar
    
    def forward(self, tj):
        spike_sum, v_sqare_sum, not_spiked_sum = 0,0.0, 0

        x, spike, v, not_spiked  = self.conv_first(tj)
        spike_sum += spike
        v_sqare_sum +=v
        not_spiked_sum += not_spiked
        x, spike = self.pool(x)

        spike_sum += spike
        # print(x.shape)
        x = x
        x, spike1, v1, not_spiked1 = self.layer0SNN(torch.concat((x, torch.ones(x.shape).to("cuda") * self.tmin_post_pool),dim=1))
        x, spike2, v2, not_spiked2 = self.layer1SNN(x)

        x, spike3, v3, not_spiked3 = self.layer2SNN(x)

        x, spike4, v4, not_spiked4 = self.layer3SNN(x)
        x = x[:512]
        # print(x[1].max(),x[1].min())
        x, spike5 = self.pool2(x)
        x = x.contiguous().view(x.size(0), -1)

        x, spike6, v5, not_spiked5 = self.layer_fc(x)

        x, spike7, v6, not_spiked6 = self.layer_fc2(x)

        spike_sum += spike1+spike2+spike3+spike4+spike5+spike6+spike7
        v_sqare_sum += v1+v2+v3+v4+v5+v6
        not_spiked_sum += not_spiked1+not_spiked2+not_spiked3+not_spiked4+not_spiked5+not_spiked6
        return x, spike_sum, v_sqare_sum, not_spiked_sum

In [9]:
model_ttfs = ResNet_ttfs(model_resnet)

c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(t_min + self.B_n*max_V+eps_V, dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:312: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(max(t_min + self.B_n*max_V+eps_V, minimal_t_max), dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:281: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_min = torch.tensor

torch.Size([1024])


C:\Users\nikos\AppData\Local\Temp\ipykernel_11116\3812665932.py:32: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3575.)
  max_vect = (torch.ones([512, 1, 1]).to("cuda").T*max_vect[:512]).T.contiguous().view(512).to("cuda")
c:\D\time_to_spike\in_torch\SNN\Check_Hardtanh.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_input = torch.concat((torch.tensor(in_ranges_max).to(self.device), torch.tensor([(1)]).to(self.device)))
c:\D\time_to_spike\in_torch\SNN\Check_Hardta

In [10]:
temp = torch.rand((1,6,224,224))
temp_ttfs = 1 - temp
# print(model_ttfs.forward(temp_ttfs))
out, spike_sum, v_sqare_sum, not_spike = model_ttfs.forward(temp_ttfs.to("cuda"))

print(spike_sum, v_sqare_sum, not_spike)
print((model_ttfs.tmax - out)*model_ttfs.scalar)

tensor(14941075, device='cuda:0') tensor(8942255., device='cuda:0', grad_fn=<AddBackward0>) tensor(26280066, device='cuda:0')
tensor([[7.6986, 0.0000, 0.0000, 0.0000, 1.7163, 0.0000, 0.3225, 2.0340, 0.0000,
         0.0000, 4.8692, 0.9483, 0.0000, 0.6109, 0.0000, 0.0000, 3.0273, 0.0000,
         2.6493, 0.0000, 0.0000, 1.4897, 0.0000, 0.0000, 0.0000, 3.4178, 4.0522,
         0.0000, 0.0000, 0.0000, 3.1711, 1.0172, 1.9574, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 3.5280, 0.0000, 0.1639, 0.0000, 0.6493, 0.0000, 0.0000,
         1.1974, 0.0000, 4.2195, 0.0000, 0.9133, 0.0000, 0.0000, 0.0000, 2.4274,
         0.0000, 2.3014, 1.2065, 0.0000, 0.0000, 1.9559, 2.8606, 4.3958, 4.2156,
         1.9866, 4.3325, 0.0000, 2.3967, 0.2875, 0.0000, 2.1941, 0.0000, 2.9444,
         0.4490, 2.0685, 7.8869, 0.7269, 0.1466, 0.0000, 0.8007, 4.0575, 1.6267,
         0.0000, 0.5855, 0.0000, 0.0000, 1.2420, 1.9181, 0.0000, 2.6100, 2.8989,
         0.0000, 0.0000, 3.8951, 3.4303, 1.1025, 0.0000, 0.0000,

In [11]:
model_resnet(temp.to("cuda"))

tensor([[ 7.6984, -0.7046, -0.6676, -0.3887,  1.7143, -2.1369,  0.3208,  2.0340,
         -4.5775, -2.9802,  4.8676,  0.9474, -4.4138,  0.6135, -5.3428, -3.1808,
          3.0216, -0.5119,  2.6507, -5.8562, -0.9628,  1.4884, -2.8162, -0.9480,
         -1.9503,  3.4153,  4.0505, -2.7961, -1.6047, -0.8419,  3.1686,  1.0198,
          1.9541, -0.9811, -0.1313, -0.8922, -1.9346, -1.8167,  3.5239, -0.9831,
          0.1649, -1.4457,  0.6468, -3.0771, -5.7645,  1.1991, -0.2768,  4.2190,
         -0.7108,  0.9107, -5.1726, -0.7129, -0.2378,  2.4241, -3.4076,  2.3002,
          1.2069, -1.6894, -1.0953,  1.9545,  2.8572,  4.3972,  4.2141,  1.9859,
          4.3343, -5.7731,  2.3984,  0.2910, -6.5573,  2.1953, -4.3225,  2.9450,
          0.4492,  2.0707,  7.8855,  0.7273,  0.1436, -3.7532,  0.8031,  4.0542,
          1.6277, -1.9374,  0.5892, -2.3227, -0.6422,  1.2416,  1.9213, -3.1069,
          2.6065,  2.9017, -5.3033, -5.2664,  3.8975,  3.4310,  1.1021, -0.4162,
         -7.2292,  1.4139, -

In [12]:
from torchvision.transforms import v2
torch.manual_seed(19)

transforms = v2.Compose([
    # v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(60),
    v2.ToDtype(torch.float32)
])


class NCaltech101ImageDataset(Dataset):
    def __init__(self, img_dir_file, transform=None, target_transform=None):
        self.images = np.load(img_dir_file + '_x.npy')
        self.labels = np.load(img_dir_file + '_y.npy')
        self.transform = transform
        self.transform = transform
        self.stage = 0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        label_temp = np.zeros((101,))
        label_temp[label] = 1
        if self.stage == 0:
            return self.transform(torch.tensor(image)), torch.tensor(label_temp)
        else:
            return torch.tensor(image), torch.tensor(label_temp)
    
    def set_stage(self, stage):
        self.stage = stage

In [13]:
from torch.utils.data import DataLoader
# training_data = NCarsImageDataset("./Datasety/nCars_train_EST_exp_", transform=transforms)
data = NCaltech101ImageDataset("./Datasety/Ncaltech101_EST_trilinear_corr", transform=transforms)


In [14]:
data.set_stage(1)
spike_acc = 0
not_spike_acc = 0
v_sqare_acc = 0.0
MAE_sum = 0
MSE_sum = 0
TP = 0
for i in tqdm(range(len(data))):#10
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            temp_data, temp_label = deepcopy(data[i])
            temp_ttfs = (1 - temp_data).unsqueeze(0).to("cuda")
            temp_data = temp_data.to("cuda")
            # print(temp_data.shape)
            # print(model_ttfs.forward(temp_ttfs))
            out, spike_sum, v_sqare_sum, not_spike = model_ttfs.forward(temp_ttfs)
            spike_acc+=spike_sum
            not_spike_acc += not_spike
            v_sqare_acc+=v_sqare_sum
            output = ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:]
            output_true = model_resnet(temp_data.unsqueeze(0))
            MAE = torch.sum((output-output_true).abs())/101.0
            MSE = torch.sqrt(torch.sum((output-output_true)*(output-output_true)))/101.0
            MAE_sum += MAE
            MSE_sum += MSE
            TP += (torch.argmax(temp_label)==torch.argmax(output))
            del temp_data, temp_ttfs
    if i%100==0:
        torch.cuda.empty_cache()
print(spike_acc/len(data))
print("Mean MAE:", MAE_sum/len(data))
print("Mean MSE:", MSE_sum/(len(data)))
print("accuracy:",float(TP)/len(data))
print("relative fraction: ", v_sqare_acc/not_spike_acc)
print("Sum V_square^2/treshold^2: ", v_sqare_acc/len(data))
print("Not spiked: ", not_spike/len(data))

  0%|          | 0/8709 [00:00<?, ?it/s]

100%|██████████| 8709/8709 [3:03:33<00:00,  1.26s/it]  

tensor(14838930., device='cuda:0')
Mean MAE: tensor(0.0067, device='cuda:0')
Mean MSE: tensor(0.0008, device='cuda:0')
accuracy: 0.5911126420943851
relative fraction:  tensor(0.3396, device='cuda:0')
Sum V_square^2/treshold^2:  tensor(8955840., device='cuda:0')
Not spiked:  tensor(3034.6699, device='cuda:0')


In [15]:
print("Not spiked: ", not_spike_acc/len(data))

Not spiked:  tensor(26372302., device='cuda:0')
